# Demo: Train a Digit Classification Model

## Step 0: Import necessary libraries
Here is the sample code of how to import necessary libraries:

In [1]:
import time
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import keras.backend as K

from ipywidgets import interact
from keras import layers, Model, optimizers
from keras.applications import VGG19
from keras.callbacks import ModelCheckpoint
from keras.utils import to_categorical


## Step 1:  Setup GPU’s memory policies and tensor’s behavior

Here is the sample code of how to setup GPU’s memory policies and tensor’s behavior:

In [2]:
physical_devices  =  tf.config.list_physical_devices('GPU')
if physical_devices:
    try:
        for gpu in physical_devices:
            tf.config.experimental.set_memory_growth(physical_devices[0], True)
            tf.experimental.numpy.experimental_enable_numpy_behavior(prefer_float32 = True)
    except RuntimeError as error:
        print(error)


## Step 2: Data Loading and Preprocessing

Here is the sample code of how to load, preprocess, and interact with the MNIST dataset:

In [3]:
def preprocess_mnist(x: object, y: object, x_new_shape: tuple) ->tuple[object, object]:
    """
    @author: Vo, Huynh Quang Nguyen

    Preprocess the MNIST dataset.

    This function preprocess_mnist preprocesses the MNIST dataset by:
    1. Upscaling the dimensions of the input images (from)
    """
    x = x.reshape(x.shape[0], 28, 28, 1).astype('float32')
    x = tf.image.resize(x, x_new_shape)
    x = np.stack((x[:,:,:,0],) * 3, axis = -1) 
    x = x.astype('float32') / 255.0
    y = to_categorical(y)

    return x, y 

(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
x_train, y_train = preprocess_mnist(x_train, y_train, (64,64))
x_test, y_test= preprocess_mnist(x_test, y_test, (64,64))

print([x_train.shape, y_train.shape])
print([x_test.shape, y_train.shape])

[(60000, 64, 64, 3), (60000, 10)]
[(10000, 64, 64, 3), (60000, 10)]


## Step 3: Define the model
Here is the sample code of how to define a model using Keras functional API:

In [4]:
def vgg19(input_shape: tuple) -> object:
    """
    @author: Vo, Huynh Quang Nguyen
    """
    inputs = layers.Input(shape = input_shape, name = 'inputs')
    vggmodel = VGG19(include_top = False, input_tensor = inputs, weights = 'imagenet')
    vggmodel.trainable = True
    x = vggmodel.output
    x = layers.GlobalAveragePooling2D(name = 'globavgpool')(x)
    x = layers.Dense(4096, activation = 'relu', name = 'dense1')(x)
    outputs = layers.Dense(10, activation = 'softmax', name = 'outputs')(x)
    model = Model(inputs = inputs, outputs = outputs, name = 'VGG19')
    
    model.compile(loss = 'categorical_crossentropy', optimizer = optimizers.Adam(learning_rate = 0.0001), 
                  metrics = ['accuracy'])
    model.summary()
    
    return model

vgg19 = vgg19((64,64,3))

Model: "VGG19"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 inputs (InputLayer)         [(None, 64, 64, 3)]       0         
                                                                 
 block1_conv1 (Conv2D)       (None, 64, 64, 64)        1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 64, 64, 64)        36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 32, 32, 64)        0         
                                                                 
 block2_conv1 (Conv2D)       (None, 32, 32, 128)       73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 32, 32, 128)       147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 16, 16, 128)       0     

## Step 4: Train the model

Here is the sample code of how to train the model using `ModelCheckpoint` from Keras:

In [5]:
def train_classification_model(model: object, model_name: str, X: object, Y: object, 
    metric_to_monitor: str, no_of_epochs: int, batch_size: int, validation_split_ratio: float) -> tuple[object, float]:
    """
    @author: Vo, Huynh Quang Nguyen
    """
    
    start_time = time.time()
    ###
    weight_path = 'weights/VGG19_{epoch:02d}_{val_accuracy:.2f}.hdf5'
    checkpoint = ModelCheckpoint(weight_path, monitor = metric_to_monitor, 
        verbose = 1, save_best_only = True, mode = 'max')
    callbacks_list = [checkpoint]
    history = model.fit(X, Y, validation_split = validation_split_ratio, epochs = no_of_epochs, 
        batch_size = batch_size, callbacks = callbacks_list, verbose = 1)
    np.save(f'{model_name}_history.npy', history.history)
    ###
    end_time = time.time()

    training_time = round(end_time - start_time, 4)
    return history, training_time

train_classification_model(vgg19, 'VGG19', x_train, y_train, 'val_accuracy', 10, 32, 0.33)

Epoch 1/10
1257/1257 [==============================] - ETA: 0s - loss: 0.1068 - accuracy: 0.9681
Epoch 1: val_accuracy improved from -inf to 0.98914, saving model to weights\VGG19_01_0.99.hdf5
1257/1257 [==============================] - 62s 46ms/step - loss: 0.1068 - accuracy: 0.9681 - val_loss: 0.0409 - val_accuracy: 0.9891
Epoch 2/10
1257/1257 [==============================] - ETA: 0s - loss: 0.0484 - accuracy: 0.9870
Epoch 2: val_accuracy improved from 0.98914 to 0.99096, saving model to weights\VGG19_02_0.99.hdf5
1257/1257 [==============================] - 57s 46ms/step - loss: 0.0484 - accuracy: 0.9870 - val_loss: 0.0355 - val_accuracy: 0.9910
Epoch 3/10
1257/1257 [==============================] - ETA: 0s - loss: 0.0334 - accuracy: 0.9917
Epoch 3: val_accuracy did not improve from 0.99096
1257/1257 [==============================] - 58s 46ms/step - loss: 0.0334 - accuracy: 0.9917 - val_loss: 0.0417 - val_accuracy: 0.9902
Epoch 4/10
1257/1257 [==============================] -

(<keras.callbacks.History at 0x1a8a7958910>, 583.7799)